## Access Ollama Server by Openai() function

#### Reference
- Structured Outputs with Ollama: https://github.com/jxnl/instructor/blob/main/docs/examples/ollama.md
- Controlling Large Language Model Output with Pydantic: https://medium.com/@mattchinnock/controlling-large-language-model-output-with-pydantic-74b2af5e79d1

In [ ]:
# !uv pip install pydantic instructor openai yfinance

In [2]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import yfinance as yf
import instructor

In [2]:
company = "Google"

class StockInfo(BaseModel):
    company: str = Field(..., description="Name of the company")
    ticker: str = Field(..., description="Ticker symbol of the company")

In [3]:
# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

resp = client.chat.completions.create(
    model="llama3",
    messages=[
        {
            "role": "user",
            "content": f"Return the company name and the ticker symbol of the {company}."
        }
    ],
    response_model=StockInfo,
    max_retries=5
)

In [39]:
stock = yf.Ticker(resp.ticker)
hist = stock.history(period="1d")
stock_price = hist['Close'].iloc[-1]
print(f"The stock price of the {resp.company} is {stock_price}. USD")

The stock price of the Google LLC is 177.85000610351562. USD


---

In [72]:
class test(BaseModel):
    date: str = Field(..., description="Date of booking")
    period: str = Field(..., description="Period of booking")

In [77]:
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="Ollama"
    ),
    mode=instructor.Mode.JSON,
)

prompt = """
Instruction:
Return the date and period based on the user input. You need to take the example as reference to answer question
if no date or no period return None

### Example
Qurestion: Any Booking available at 2/2/2024
return: {'date'='2-2-2024', 'period'='None'}

### Example of irrelevant query
Qurestion: Good morning
return: {'date'='None', 'period'='None'}

Query:
I want to book service
"""

response = client.chat.completions.create(
    model="llama3",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    response_model=test,
    max_retries=5,
)

response

test(date='None', period='None')

---

In [27]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

In [24]:
llm = ChatOllama(model="llama3",
                 keep_alive=-1,
                 temperature=0,
                 max_new_tokens=512)

In [70]:
content = """
You are a helpful agent for company to handle booking room with client.
You need to help client finish the booking progress by attaining a correct date and period from the customer
Please follow the example to guide client to respond

### Example: unclear date and period
Client: I want to book room
AI: what is the date and period for the room?

### Example: unclear date
Client: I want to book next friday
AI: Please provide me a clear date e.g. 2023/05/02

Query:
Hi
"""

messages = [
    HumanMessage(
        content=content
    )
]

chat_model_response = llm.invoke(messages)
print(chat_model_response.content)

Hello! Thank you for reaching out to us. To proceed with booking a room, could you please provide me with a specific date and period? For example, what dates are you looking at (start and end dates)? This will help me assist you better.


In [26]:
prompt = ChatPromptTemplate.from_template("Write me a 50 word article on {topic}")
chain = prompt | llm
print(chain.invoke({"topic":"LLMs"}))

content='Here is a 50-word article on Large Language Models (LLMs):\n\nLarge Language Models (LLMs) are artificial intelligence systems that can process and understand vast amounts of text data. These models, such as BERT and RoBERTa, have revolutionized natural language processing by enabling applications like language translation, sentiment analysis, and text summarization. LLMs learn from massive datasets and can generate human-like responses, making them a game-changer for industries like customer service, marketing, and healthcare.' response_metadata={'model': 'llama3', 'created_at': '2024-05-23T03:51:12.386885257Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4483107130, 'load_duration': 2651963075, 'prompt_eval_count': 21, 'prompt_eval_duration': 100555000, 'eval_count': 102, 'eval_duration': 1679018000} id='run-61a345a8-c37c-43d9-a872-49ebf5deb24f-0'
